In [78]:
from datetime import date, datetime


def is_a_stock_trading_day(check_date):
    """
    判断单个日期是否有东方财富财经早餐（即是否为A股交易日）
    :param check_date: 待判断的单个日期（date或datetime对象）
    :return: True（有财经早餐）/False（无）；超出2022.11-2025范围返回None
    """
    # 处理输入类型：若为datetime，提取date部分
    if isinstance(check_date, datetime):
        check_date = check_date.date()

    # 检查日期范围（2022-11-01 至 2025-12-31）
    valid_start = date(2022, 11, 1)
    valid_end = date(2025, 12, 31)
    if not (valid_start <= check_date <= valid_end):
        return None  # 超出范围

    # 1. 周末（周六/周日）无财经早餐（A股休市）
    weekday = check_date.weekday()  # 0=周一，5=周六，6=周日
    if weekday in (5, 6):
        return False

    # 2. 各年份休市日期（同之前的整理，非交易日无财经早餐）
    closed_days = {
        2022: [date(2022, 12, 31)],  # 2022年11-12月休市日
        2023: [
            date(2023, 1, 1), date(2023, 1, 2),  # 元旦
            date(2023, 1, 21), date(2023, 1, 22), date(2023, 1, 23),
            date(2023, 1, 24), date(2023, 1, 25), date(2023, 1, 26), date(2023, 1, 27),  # 春节
            date(2023, 4, 5),  # 清明节
            date(2023, 4, 29), date(2023, 4, 30), date(2023, 5, 1),
            date(2023, 5, 2), date(2023, 5, 3),  # 劳动节
            date(2023, 6, 22), date(2023, 6, 23), date(2023, 6, 24),  # 端午节
            date(2023, 9, 29), date(2023, 9, 30), date(2023, 10, 1),
            date(2023, 10, 2), date(2023, 10, 3), date(2023, 10, 4),
            date(2023, 10, 5), date(2023, 10, 6)  # 中秋+国庆
        ],
        2024: [
            date(2024, 1, 1),  # 元旦
            date(2024, 2, 10), date(2024, 2, 11), date(2024, 2, 12),
            date(2024, 2, 13), date(2024, 2, 14), date(2024, 2, 15),
            date(2024, 2, 16), date(2024, 2, 17),  # 春节
            date(2024, 4, 4),  # 清明节
            date(2024, 5, 1), date(2024, 5, 2), date(2024, 5, 3),
            date(2024, 5, 4), date(2024, 5, 5),  # 劳动节
            date(2024, 6, 10),  # 端午节
            date(2024, 9, 17),  # 中秋节
            date(2024, 10, 1), date(2024, 10, 2), date(2024, 10, 3),
            date(2024, 10, 4), date(2024, 10, 5), date(2024, 10, 6), date(2024, 10, 7)  # 国庆
        ],
        2025: [
            date(2025, 1, 1),  # 元旦
            date(2025, 1, 28), date(2025, 1, 29), date(2025, 1, 30),
            date(2025, 1, 31), date(2025, 2, 1), date(2025, 2, 2), date(2025, 2, 3),  # 春节
            date(2025, 4, 4),  # 清明节
            date(2025, 5, 1), date(2025, 5, 2), date(2025, 5, 3),
            date(2025, 5, 4), date(2025, 5, 5),  # 劳动节
            date(2025, 6, 1), date(2025, 6, 2),  # 端午节
            date(2025, 9, 8),  # 中秋节
            date(2025, 10, 1), date(2025, 10, 2), date(2025, 10, 3),
            date(2025, 10, 4), date(2025, 10, 5), date(2025, 10, 6), date(2025, 10, 7)  # 国庆
        ]
    }

    # 3. 检查是否在当年休市列表中（休市日无财经早餐）
    year = check_date.year
    if check_date in closed_days[year]:
        return False

    # 否则为交易日，有财经早餐
    return True


In [36]:
from datetime import date,datetime,timedelta
def get_adjusted_workday(date_input = None) -> date:
    """
    根据规则获取调整后的工作日（仅返回年月日的date类型）：
    - 未输入日期或输入日期是当日：用当前时间判断，早于10点则基准为前一天，否则为当天；
    - 输入日期是历史日期（非当日）：忽略具体时间，仅用其年月日作为基准；
    - 基准日期为工作日则返回，否则返回上一个工作日（仅含年月日）。
    """
    today = datetime.now().date()
    now = datetime.now()

    if date_input is None:
        base_date = (now - timedelta(days=1)).date() if now.hour < 10 else today
    else:
        input_date = date_input.date() if isinstance(date_input, datetime) else date_input
        if input_date == today:
            base_date = (now - timedelta(days=1)).date() if now.hour < 10 else today
        else:
            base_date = input_date

    if is_a_stock_trading_day(base_date):
        return base_date
    else:
        current_date = base_date - timedelta(days=1)
        while not is_a_stock_trading_day(current_date):
            current_date -= timedelta(days=1)
        return current_date


In [85]:
import pandas as pd
from datetime import datetime
specific_date="2022-12-22"
last_workday =datetime.strptime(specific_date,"%Y-%m-%d").date()
end_date=get_adjusted_workday(last_workday)
print(end_date)
today=datetime.now().date()
start_date=date(year=2022,month=11,damy=9)
if last_workday < start_date:
    last_workday=start_date

# 计算工作日总数
date_range= pd.date_range(start=end_date, end=today)
workday_count = sum(1 for d in date_range if is_a_stock_trading_day(d.date()))
print(workday_count)
page= (workday_count-1) // 20 +1
print(page)

2022-12-22
700
35
